In [1]:
from utils import *

import os

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [45]:
# Misc
LOG_DIR = './log_pt/'
MODEL_ID = 'storkes_plain'

# Training Parameters
learning_rate = 0.001
num_epochs = 200
batch_size = 256
display_step = 10

# Network Parameters
num_classes = 5
keep_rate = 1

repeat_size = 82
crop_size = 78

if not os.path.isdir(LOG_DIR):
    os.makedirs(LOG_DIR)

In [46]:
raw = pd.read_csv('./storks_obs_train.csv', header=None)

In [47]:
mean = np.array([0. for i in  range(3)])
X = []
Y = []

for i in range(len(raw)):
    
    row = np.array(raw.loc[i][~raw.loc[i].isnull()])
    data = row[0:-1].astype(float)
    label = row[-1]
    
    x_axis = data[0::3]
    y_axis = data[1::3]
    z_axis = data[2::3]
    
    x_axis = repeat_crop_data (x_axis, repeat_size)
    y_axis = repeat_crop_data (y_axis, repeat_size)
    z_axis = repeat_crop_data (z_axis, repeat_size)
    
    mean[0] += x_axis.mean()
    mean[1] += y_axis.mean()
    mean[2] += z_axis.mean()
    
    X.append(np.stack((x_axis, y_axis, z_axis)))
    Y.append(label)

mean = mean/len(raw)
    
X = np.array(X).transpose((0, 2, 1))
Y = np.array(Y)
label_names = np.unique(Y)

le = preprocessing.LabelEncoder()
le.fit(np.unique(Y))
Y = le.transform(Y)

In [48]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.1)

In [49]:
class acc_dataset_augment (torch.utils.data.Dataset):
    
    def __init__ (self, X, Y, mean, input_size, crop_size):
        super().__init__()
        self.X = X
        self.Y = Y.astype(np.float)
        self.mean = mean
        self.input_size = input_size
        self.crop_size = crop_size
        
    def __len__ (self):
        return len(self.X)
    
    def __getitem__ (self, idx):
        
        x = self.X[idx]
        y = self.Y[idx]
        
        x = (x - self.mean).transpose(1, 0)
        
        start_idx = np.random.randint(self.input_size - self.crop_size, size=1).item()
        
        x_axis = x[0, start_idx:start_idx+self.crop_size]
        y_axis = x[1, start_idx:start_idx+self.crop_size]
        z_axis = x[2, start_idx:start_idx+self.crop_size]
        
        x = np.stack((x_axis, y_axis, z_axis))
            
        return torch.tensor(x), torch.tensor(y).type(torch.LongTensor)

In [50]:
train_set = acc_dataset_augment(train_x, train_y, mean, input_size=repeat_size, crop_size=crop_size)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

test_set = acc_dataset_augment(test_x, test_y, mean, input_size=repeat_size, crop_size=crop_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

In [51]:
class MyNet (nn.Module):
    
    def __init__ (self, num_classes, input_dim, drop_rate):
        super().__init__()
        
        self.input_dim = input_dim
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc3 = nn.Linear(self.input_dim*64, 1024)
        self.drop = nn.Dropout(p=drop_rate)
        self.fc4 = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        #self.pretrained_resnet.avgpool(x)
        x = x.view(-1, self.input_dim*64)
        
        x = F.relu(self.fc3(x))
        x = self.drop(x)
        x = self.fc4(x)

        return x

In [52]:
net = MyNet(num_classes=num_classes, input_dim=crop_size, drop_rate=0.3).double()

net.to(device)

MyNet(
  (conv1): Conv1d(3, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc3): Linear(in_features=4992, out_features=1024, bias=True)
  (drop): Dropout(p=0.3)
  (fc4): Linear(in_features=1024, out_features=5, bias=True)
)

In [53]:
# Loss function
loss_function = nn.CrossEntropyLoss().double()

# Optimizer
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# Decay LR by a factor of 0.1 every 30 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [54]:
step = 0

net.train()

for epoch in range(num_epochs):
    
    exp_lr_scheduler.step()
    
    for data in train_loader:
        
        x, y = data
        
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        
        logits = net(x)
        
        loss = loss_function(logits, y)
        
        loss.backward()
        
        optimizer.step()
        
        step += 1
        
        _, preds = torch.max(logits, 1)
        
        correct = (preds == y).sum().item()
        
        accuracy = correct/len(x)
        
        if step % display_step == 0:
            print('Epoch: %d, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (epoch, step, loss, accuracy))
            
print('Final, Step: %d, Minibatch loss: %.3f, Minibatch accuracy: %.3f' % 
                  (step, loss, accuracy))


Epoch: 1, Step: 10, Minibatch loss: 0.841, Minibatch accuracy: 0.730
Epoch: 2, Step: 20, Minibatch loss: 0.657, Minibatch accuracy: 0.801
Epoch: 4, Step: 30, Minibatch loss: 0.579, Minibatch accuracy: 0.809
Epoch: 5, Step: 40, Minibatch loss: 0.452, Minibatch accuracy: 0.855
Epoch: 7, Step: 50, Minibatch loss: 0.430, Minibatch accuracy: 0.863
Epoch: 8, Step: 60, Minibatch loss: 0.360, Minibatch accuracy: 0.883
Epoch: 9, Step: 70, Minibatch loss: 0.237, Minibatch accuracy: 0.914
Epoch: 11, Step: 80, Minibatch loss: 0.209, Minibatch accuracy: 0.941
Epoch: 12, Step: 90, Minibatch loss: 0.259, Minibatch accuracy: 0.930
Epoch: 14, Step: 100, Minibatch loss: 0.186, Minibatch accuracy: 0.945
Epoch: 15, Step: 110, Minibatch loss: 0.263, Minibatch accuracy: 0.910
Epoch: 17, Step: 120, Minibatch loss: 0.241, Minibatch accuracy: 0.926
Epoch: 18, Step: 130, Minibatch loss: 0.195, Minibatch accuracy: 0.949
Epoch: 19, Step: 140, Minibatch loss: 0.378, Minibatch accuracy: 0.914
Epoch: 21, Step: 150, 

In [44]:
net.eval()

test_correct = 0

with torch.no_grad():
    
    for data in test_loader:

        test_x, test_y = data
        
        test_x, test_y = test_x.to(device), test_y.to(device)

        test_logits = net(test_x)
        _, test_preds = torch.max(test_logits, 1)

        test_correct_batch = (test_preds == test_y).sum().item()

        test_correct += test_correct_batch

print('Test accuracy: %.3f' % (test_correct / len(test_set)))

Test accuracy: 0.914


In [31]:
torch.save(net.state_dict(), os.path.join(LOG_DIR, 'model_%s_checkpoint.pth.tar' % 'storks_plain_ep_100_step_20'))